# K8S Resource Test


In [23]:
SCHEDULER_IP=!kubectl get svc seldon-scheduler -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
SCHEDULER_IP=SCHEDULER_IP[0]
import os
os.environ['SCHEDULER_IP'] = SCHEDULER_IP

In [24]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP

## Show ServerConfigurations Available

In [25]:
!kubectl get serverconfig -n seldon-mesh

NAME       AGE
mlserver   16s
triton     16s


In [26]:
!kubectl get serverconfig -n seldon-mesh mlserver -o jsonpath='{.spec}' | jq .

{
  "podSpec": {
    "containers": [
      {
        "image": "seldonio/seldon-rclone:latest",
        "imagePullPolicy": "IfNotPresent",
        "name": "rclone",
        "ports": [
          {
            "containerPort": 5572,
            "name": "http",
            "protocol": "TCP"
          }
        ],
        "readinessProbe": {
          "failureThreshold": 3,
          "initialDelaySeconds": 5,
          "periodSeconds": 5,
          "successThreshold": 1,
          "tcpSocket": {
            "port": 5572
          },
          "timeoutSeconds": 1
        },
        "volumeMounts": [
          {
            "mountPath": "/mnt/agent",
            "name": "mlserver-models"
          }
        ]
      },
      {
        "env": [
          {
            "name": "SELDON_SERVER_CAPABILITIES",
            "value": "sklearn,xgboost"
          },
          {
            "name": "SELDON_OVERCOMMIT",
            "value": "false"
          },
          {
            "name": "SELDON_SERVE

## Deploy MLServer Instance

In [27]:
!cat ../../operator/samples/servers/mlserver1.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Server
metadata:
  name: mlserver1
  namespace: seldon-mesh
spec:
  server:
    type: mlserver


In [28]:
!kubectl create -f ../../operator/samples/servers/mlserver1.yaml

server.mlops.seldon.io/mlserver1 created


In [29]:
!kubectl wait --for condition=ready --timeout=300s server --all -n seldon-mesh

server.mlops.seldon.io/mlserver1 condition met


## Deploy Model

In [30]:
!cat ../../operator/samples/models/sklearn-iris-gs.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Model
metadata:
  name: iris
  namespace: seldon-mesh
spec:
  storageUri: "gs://seldon-models/mlserver/iris"
  artifactVersion: 1
  requirements:
  - sklearn
  replicas: 1


In [31]:
!kubectl create -f ../../operator/samples/models/sklearn-iris-gs.yaml

model.mlops.seldon.io/iris created


In [32]:
!kubectl wait --for condition=ready --timeout=300s model --all -n seldon-mesh

model.mlops.seldon.io/iris condition met


In [33]:
!kubectl get model iris -n seldon-mesh -o jsonpath='{.status}' | jq .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-01-08T11:49:03Z",
      "status": "True",
      "type": "ModelReady"
    },
    {
      "lastTransitionTime": "2022-01-08T11:49:03Z",
      "status": "True",
      "type": "Ready"
    }
  ]
}


In [34]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x558cdadb54f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x558cdadb54f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: iris
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Sat, 08 Jan 2022 11:49:05 GMT
< server: envoy
< content-length: 194
< content-type: application/json
< x-envoy-upstream-service-time: 859
< 
* Connection #0 to host 172.18.255.3 left intact
{"model_name":"iris","model_version":"1","id":"70be983f-5221-4968-bfff-acb9acff18a7","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [35]:
!kubectl get server mlserver1 -n seldon-mesh -o jsonpath='{.status}' | jq .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-01-08T11:48:44Z",
      "status": "True",
      "type": "Ready"
    },
    {
      "lastTransitionTime": "2022-01-08T11:48:44Z",
      "reason": "StatefulSet replicas matches desired replicas",
      "status": "True",
      "type": "StatefulSetReady"
    }
  ],
  "loadedModels": 1
}


## Deploy Second Server

In [36]:
!cat ../../operator/samples/servers/mlserver2.yaml

apiVersion: mlops.seldon.io/v1alpha1
kind: Server
metadata:
  name: mlserver2
  namespace: seldon-mesh
spec:
  server:
    type: mlserver


In [37]:
!kubectl create -f ../../operator/samples/servers/mlserver2.yaml

server.mlops.seldon.io/mlserver2 created


In [38]:
!kubectl wait --for condition=ready --timeout=300s server --all -n seldon-mesh

server.mlops.seldon.io/mlserver1 condition met
server.mlops.seldon.io/mlserver2 condition met


## Delete First Server

In [39]:
!kubectl delete -f ../../operator/samples/servers/mlserver1.yaml

server.mlops.seldon.io "mlserver1" deleted


In [40]:
!kubectl get server mlserver2 -n seldon-mesh -o jsonpath='{.status}' | jq .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-01-08T11:49:37Z",
      "status": "True",
      "type": "Ready"
    },
    {
      "lastTransitionTime": "2022-01-08T11:49:37Z",
      "reason": "StatefulSet replicas matches desired replicas",
      "status": "True",
      "type": "StatefulSetReady"
    }
  ],
  "loadedModels": 1
}


In [41]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x55ce33eda4f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x55ce33eda4f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: iris
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Sat, 08 Jan 2022 11:49:52 GMT
< server: envoy
< content-length: 194
< content-type: application/json
< x-envoy-upstream-service-time: 854
< 
* Connection #0 to host 172.18.255.3 left intact
{"model_name":"iris","model_version":"2","id":"305b5bcb-f2ad-4556-af91-08fa005393df","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [45]:
!kubectl delete -f ../../operator/samples/models/sklearn-iris-gs.yaml

model.mlops.seldon.io "iris" deleted


In [46]:
!kubectl get server mlserver2 -n seldon-mesh -o jsonpath='{.status}' | jq .

{
  "conditions": [
    {
      "lastTransitionTime": "2022-01-08T11:49:37Z",
      "status": "True",
      "type": "Ready"
    },
    {
      "lastTransitionTime": "2022-01-08T11:49:37Z",
      "reason": "StatefulSet replicas matches desired replicas",
      "status": "True",
      "type": "StatefulSetReady"
    }
  ],
  "loadedModels": 0
}


In [47]:
!kubectl delete -f ../../operator/samples/servers/mlserver2.yaml

server.mlops.seldon.io "mlserver2" deleted


## Debug

In [42]:
!grpcurl -d '{"model":{"name":"iris"},"allVersions":true}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "versions": [
    {
      "version": 1,
      "serverName": "mlserver1",
      "kubernetesMeta": {
        "namespace": "seldon-mesh",
        "generation": "1"
      },
      "modelReplicaState": {
        "0": {
          "state": "Unloaded",
          "lastChangeTimestamp": "2022-01-08T11:49:48.226100917Z"
        }
      },
      "state": {
        "state": "ModelTerminated",
        "lastChangeTimestamp": "2022-01-08T11:49:48.226100917Z"
      }
    },
    {
      "version": 2,
      "serverName": "mlserver2",
      "kubernetesMeta": {
        "namespace": "seldon-mesh",
        "generation": "1"
      },
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-01-08T11:49:48.114861383Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-01-08T11:49:48.114861383Z"
      }
    }
  ]
}


In [43]:
!grpcurl -d '{"name":"mlserver1"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

ERROR:
  Code: FailedPrecondition
  Message: Failed to find server mlserver1


In [44]:
!grpcurl -d '{"name":"mlserver2"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver2",
  "resources": [
    {
      "memory": "1000000000",
      "availableMemoryBytes": "1000000000",
      "numLoadedModels": 1
    }
  ],
  "expectedReplicas": 1,
  "availableReplicas": 1,
  "numLoadedModels": 1,
  "kubernetesMeta": {
    "namespace": "seldon-mesh",
    "generation": "2"
  }
}
